# "European Covid data exploration"
> "Exploring which countries have had the highest and lowest covid numbers in Europe"

- toc: true
- branch: master
- badges: true
- comments: false
- author: Ifan Johnston
- categories: [covid]

# Importing and preparing the data

We will be looking at data from the following countries:

- Italy
- Austria
- Germany
- Belgium
- France
- United Kingdom
- Portugal

We begin by importing the data, and adding some new features so that we can compare the data from different countries. For example we calculate 'confirmed cases per 100k population', 'deaths per 100k' and 'new cases' since these are not initially in the dataset.

In [1]:
#collapse
from covid19dh import covid19
import altair as alt
import datetime

countries = ["Italy", 
             "Austria",
             "Germany",
             "Belgium",
             "France",
             "United Kingdom",
             "Switzerland",
             "Portugal"
            ]

yesterday = datetime.date.today() - datetime.timedelta(days=1)

x, src = covid19(countries, raw=True, verbose=False, end=yesterday, cache=False)

x_small = x.loc[:, ['administrative_area_level_1', 'date', 'vaccines', 'confirmed','tests', 'recovered', 'deaths', 'population']]
x_small.rename(columns={'administrative_area_level_1': 'id'}, inplace=True)

x_small['confirmed_per'] = 100000 * x_small['confirmed'] / x_small['population']
x_small['deaths_per'] = 100000 * x_small['deaths'] / x_small['population']
x_small['ratio'] = 100 * (x_small['deaths']) / (x_small['confirmed'])
x_small['tests_per'] = 100000 * (x_small['tests']) / (x_small['population'])
x_small['vaccines_per'] = x_small['vaccines'] / x_small['population']

x_small['new_cases']=x_small.groupby('id').confirmed.diff().fillna(0)
x_small['new_cases_per']=x_small.groupby('id').confirmed_per.diff().fillna(0)
# x_small.fillna(0, inplace=True)
x_small.to_csv("../data_sets/european_covid.csv", index=False)
x_small_url = "../data_sets/european_covid.csv"

Here is a random sample of 5 rows from the dataset.

In [2]:
x_small.query("id=='Belgium' & date >= '2021-07-01'").head(10)

,id,date,vaccines,confirmed,tests,recovered,deaths,population,confirmed_per,deaths_per,ratio,tests_per,vaccines_per,new_cases,new_cases_per
9739,Belgium,2021-07-01,11188825.0,1089644.0,15445339.0,NaN,25186.0,11433256.0,9530.478457,220.287204,2.311397,135091.342309,0.978621,676.0,5.912576
9740,Belgium,2021-07-02,11303237.0,1090368.0,15527027.0,NaN,25189.0,11433256.0,9536.810861,220.313444,2.310137,135805.819445,0.988628,724.0,6.332404
9741,Belgium,2021-07-03,11392304.0,1090889.0,15583057.0,NaN,25192.0,11433256.0,9541.367743,220.339683,2.309309,136295.881068,0.996418,521.0,4.556882
9742,Belgium,2021-07-04,11406670.0,1091255.0,15616965.0,NaN,25194.0,11433256.0,9544.568931,220.357176,2.308718,136592.454503,0.997675,366.0,3.201188
9743,Belgium,2021-07-05,11527185.0,1092252.0,15670788.0,NaN,25197.0,11433256.0,9553.289107,220.383415,2.306885,137063.212789,1.008215,997.0,8.720176
9744,Belgium,2021-07-06,11642884.0,1093295.0,15730981.0,NaN,25198.0,11433256.0,9562.411617,220.392161,2.304776,137589.685738,1.018335,1043.0,9.122511
9745,Belgium,2021-07-07,11802236.0,1094467.0,15796271.0,NaN,NaN,11433256.0,9572.662416,NaN,NaN,138160.739163,1.032273,1172.0,10.250798
9746,Belgium,2021-07-08,11967581.0,1095694.0,15872967.0,NaN,25200.0,11433256.0,9583.394267,220.409654,2.299912,138831.554196,1.046734,1227.0,10.731851
9747,Belgium,2021-07-09,12115482.0,1096982.0,15946094.0,NaN,25202.0,11433256.0,9594.659649,220.427147,2.297394,139471.153274,1.059670,1288.0,11.265382
9748,Belgium,2021-07-10,12244358.0,1097986.0,16007167.0,NaN,25207.0,11433256.0,9603.441050,220.470879,2.295749,140005.323068,1.070942,1004.0,8.781401


# Plotting the data

We will first look at the total numbers of cases and deaths in each country, before moving on to cases and deaths per 100k population.

{% include info.html text="In each of the charts below, you can click on the legend to filter the lines shown" %}

## Total cases per 100,000

In [3]:
#collapse

leg_selection = alt.selection_multi(fields=['id'], bind='legend')

alt.Chart(x_small_url).mark_line().encode(
    x=alt.X("yearmonthdate(date):T", axis=alt.Axis(title='Date')),
    y=alt.Y("confirmed_per:Q", axis=alt.Axis(title='Confirmed per 100k')),
    tooltip=['id:N', 'confirmed_per:Q'],
    color=alt.Color('id:N', legend=alt.Legend(title="Countries")),
    opacity=alt.condition(leg_selection, alt.value(1), alt.value(0.2))
).add_selection(leg_selection).properties(title='Total number of cases per 100,000 population for selected European Countries', width=600).interactive()


alt.Chart(...)

## Total deaths per 100,000

In [4]:
#collapse
alt.Chart(x_small_url).mark_line().encode(
    x=alt.X("yearmonthdate(date):T", axis=alt.Axis(title='Date')),
    y=alt.Y("deaths_per:Q", axis=alt.Axis(title='Deaths per 100k'), impute=alt.ImputeParams(value=50)),
    tooltip='id:N',
    color=alt.Color('id:N', legend=alt.Legend(title="Countries")),
    opacity=alt.condition(leg_selection, alt.value(1), alt.value(0.2))
).add_selection(leg_selection).properties(title='Number of deaths per 100,000 population for selected European Countries', width=600).interactive()

alt.Chart(...)

## Two week incidence rate

In [5]:
#collapse
brush = alt.selection(type='interval', encodings=['x'])

base = alt.Chart(x_small_url).mark_line().transform_window(
    rolling_mean='sum(new_cases_per)',
    frame=[-7, 0],
    groupby=['id:N']
).encode(
    x=alt.X("yearmonthdate(date):T",
            axis=alt.Axis(title='Date')
           ),
    y=alt.Y("rolling_mean:Q",
            axis=alt.Axis(title='Incidence rate')
           ),
    tooltip=['id:N', 'rolling_mean:Q'],
    color=alt.Color('id:N', legend=alt.Legend(title="Countries")),
    opacity=alt.condition(leg_selection, alt.value(1), alt.value(0.2))
).add_selection(leg_selection).properties(
    width=600,
    height=400,
    title='Number of new cases per 100,000 over two weeks for selected countries'
)

upper = base.encode(
    alt.X('yearmonthdate(date):T',axis=alt.Axis(title='Date'),
          scale=alt.Scale(domain=brush))
)

lower = base.properties(
    height=60
).add_selection(brush)

upper & lower

alt.VConcatChart(...)

The ratio of confirmed cases and deaths gives an indication of what the case fatality rate is - it seems to be between 2 and 3%, assuming that the countries listed here are catching all positive cases (which they probably aren't, so it's likely lower than this).

## Case fatality rate

In [6]:
#collapse
base = alt.Chart(x_small_url).mark_line().encode(
    x=alt.X("yearmonthdate(date):T", axis=alt.Axis(title='Date')),
    y=alt.Y("ratio:Q", axis=alt.Axis(title='Ratio of deaths per case')),
    tooltip='id:N',
    color=alt.Color('id:N', legend=alt.Legend(title="Countries")),
opacity=alt.condition(leg_selection, alt.value(1), alt.value(0.2))
).add_selection(leg_selection).properties(title='The ratio of deaths to confirmed cases (case fatality rate)', width=600)

upper = base.encode(
    alt.X('yearmonthdate(date):T',axis=alt.Axis(title='Date'),
          scale=alt.Scale(domain=brush))
)

lower = base.properties(
    height=60
).add_selection(brush)

upper & lower

alt.VConcatChart(...)

# Vaccines

In the chart below we plot the number vaccines given per population - this means that if the number is 1, then the country has given the equivalent of 1 shot for each person in the country. Since not everyone in the countries are eligible to get the vaccine, a ratio of 1 means that many people have recieved two jabs. Note also that some kinds of vaccines (the J&J's [Janssen vaccine](https://www.cdc.gov/coronavirus/2019-ncov/vaccines/different-vaccines/janssen.html), for example) only require 1 shot so the goal is not neccesarily to reach exactly 2 shots per person in the whole country.

In [7]:
#collapse
alt.Chart(x_small.query("vaccines > 0")).mark_line().encode(
    x=alt.X("yearmonthdate(date):T", axis=alt.Axis(title='Date')),
    y=alt.Y("vaccines_per:Q", axis=alt.Axis(title='Number of vaccines given')),
    tooltip=['id:N', 'vaccines_per'],
    color=alt.Color('id:N', legend=alt.Legend(title="Countries")),
    opacity=alt.condition(leg_selection, alt.value(1), alt.value(0.2))
).add_selection(leg_selection).properties(title='Number of vaccines given', width=600).interactive()

alt.Chart(...)